In [85]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Bring biomass data into dataframe

In [86]:
path = 'D:\GitHubRepos\ApsimX\Tests\Validation\Maize\Observations'
Observed = pd.read_excel(path+'\obsDataMaize_DB.xlsx',sheetname='HarvestData')
ObservedTimeSeries = pd.read_excel(path+'\obsDataMaize_DB.xlsx',sheetname='TimeSeriesData')
NZObserved = pd.read_excel(path+'\obsDataMaize_DB.xlsx',sheetname='observed')
RainShelterN = pd.read_excel(path+'\obsDataMaize_DB.xlsx',sheetname='RainShelterNitrogen')
RainShelterSW = pd.read_excel(path+'\SoilWater.xlsx',sheetname='SoilWater')
RainShelterCob = pd.read_excel(path+'\obsDataMaize_DB.xlsx',sheetname='RachisHusk_RainS')
PhenologySimple = pd.read_excel(path+'\obsDataMaize_DB.xlsx',sheetname='phenologySimple_RainS')
RainShelterSen = pd.read_excel(path+'\LeafSenesCount.xlsx',sheetname='EndSenes')
RainShelterSenStart = pd.read_excel(path+'\LeafSenesCount.xlsx',sheetname='StartSenes')
RainShelterLeafSize = pd.read_excel(path+'\IndLeafData_MaizeRainshelter.xlsx',sheetname='MaxLeafSize')

In [87]:
AllDataTabs = [Observed, ObservedTimeSeries, NZObserved, RainShelterN, RainShelterSW,
              RainShelterCob, PhenologySimple,RainShelterSen, RainShelterSenStart, RainShelterLeafSize]
               

## Make a column index

In [88]:
ColumnIndex = Observed.columns
for Table in AllDataTabs:
    ColumnIndex = ColumnIndex  | Table.columns  # the union of the merged data frame and the next in the list
ColumnIndex = ColumnIndex.drop(['SimulationName','Clock.Today'])

## Make row index

In [89]:
RowIndexData = Observed.ix[:,Observed.columns.isin(['SimulationName','Clock.Today'])]
for Table in AllDataTabs:
    RowIndexData = pd.concat([RowIndexData,Table.ix[:,Table.columns.isin(['SimulationName','Clock.Today'])]],axis=0)
RowIndex = RowIndexData.drop_duplicates()
RowIndex = RowIndex.dropna()
RowIndex.ix[:,'Clock.Today'] = pd.to_datetime(RowIndex.ix[:,'Clock.Today'])
RowIndex.set_index(['SimulationName','Clock.Today'],inplace=True)

## Make empty Data frame

In [90]:
ObsDataFrame = pd.DataFrame(index=RowIndex.index, columns=ColumnIndex)

## Set indexes in the smaller tables and deal to duplicate rows

In [ ]:
from collections import Counter
def MakeMean(ser):
    Series = pd.Series(index=ser.columns)
    for Group in ser.columns:
        data = ser[Group].dropna()
        if len(data) > 0:
            if isinstance(data.values[0], float):
                Series[Group] = data.mean()
            else:
                if isinstance(data.values[0], np.int64):
                    Series[Group] = np.round(data.mean(),0)
                else:
                    Series[Group] = Counter(data).most_common(1)[0][0]
    return Series    

In [ ]:
CompressedData = []
for Table in AllDataTabs:
    Table.ix[:,'Clock.Today'] = pd.to_datetime(Table.ix[:,'Clock.Today'])
    Table.set_index(['SimulationName','Clock.Today'],inplace=True)
    Compressed = Table.groupby(level=['SimulationName','Clock.Today']).apply(MakeMean)
    CompressedData.append(Compressed)

In [ ]:
for DataTable in range(len(CompressedData)):
    ObsDataFrame.update(CompressedData[DataTable])

In [ ]:
ObsDataFrame.where(ObsDataFrame!='*',other=np.nan,inplace=True)

In [ ]:
ObsDataFrame.to_excel(path + '\MaizeObservations.xlsx',merge_cells=False,sheet_name='Observed')